In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Input and output video file paths
input_video_path = 'Input vedio file path'
output_video_path = 'output vedio file path'

# Define the landmark numbers corresponding to the given coordinates
landmark_numbers = [
    [103, 104, 105, 66, 107, 9, 336, 296, 334, 333, 332, 297, 338, 10, 109, 67],  # Coordinates for the first polygon
    [198, 233, 111, 187],  # Coordinates for the second polygon
    [412, 371, 411, 340]   # Coordinates for the third polygon
]

# Open input video file
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# This is for Vedio Properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# codec for creating vedio with ROI
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize MediaPipe Face Mesh
with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        # Read frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # This is to Convert the frame from BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Processing the frame with MediaPipe Face Mesh
        results = face_mesh.process(frame_rgb)

        # Drawing the face mesh landmarks on the frame that is generated
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Extracting landmark coordinates for each polygon
                for polygon_landmarks in landmark_numbers:
                    polygon_coords = [(int(face_landmarks.landmark[landmark_number].x * frame_width),
                                       int(face_landmarks.landmark[landmark_number].y * frame_height))
                                      for landmark_number in polygon_landmarks]
                    pts = np.array(polygon_coords, np.int32)
                    pts = pts.reshape((-1, 1, 2))
                    cv2.polylines(frame, [pts], isClosed=True, color=(255, 0, 0), thickness=2)

        # Write the processed frame to the output video
        out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
